In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' # for using only cpu
import tensorflow as tf
import numpy as np
tf.__version__

'1.7.0'

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz


In [4]:
def my_cnn(features, labels, mode):
    # Model Architecture(Network)
    reshape_images = tf.reshape(features, [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(reshape_images, 32, [3, 3], activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, [3, 3], [2, 2])

    conv2 = tf.layers.conv2d(pool1, 64, [3, 3], activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(conv2, [3, 3], [2, 2])

    flatten_feature = tf.contrib.layers.flatten(pool2)
    dense1 = tf.layers.dense(flatten_feature, 1024, activation=tf.nn.relu)
    drop1 = tf.layers.dropout(dense1, 0.5, training = (mode == tf.estimator.ModeKeys.TRAIN))

    logits = tf.layers.dense(drop1, 10, activation=None)
    
    predictions = {
        'classes': tf.argmax(logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    } # for log
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        LEARNING_RATE = 1e-4
        opt = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=opt)
    
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# tf.layers method의 parameter

tf.layers.conv2d(input_vector, output_channel, [kernel_width, kernel_height], ...)

tf.layers.max_pooling2d(input_vector, [kernel_width, kernel_height], [strides_along_width, strides_along_height], ...)

tf.layers.dense(input_vector, output_channel, ...)

tf.layers.dropout(input_vector, drop_ratio, training, ...)

위의 기본적인 함수 외에도 Batch normalization이나 몇 가지를 매우 간편하게 제공한다. (BN은 확실히 tf.nn 보다 편하다)

최근에는 TF 공식 다큐먼트 튜토리얼에서  tf.nn이 아닌 tf.layers를 통한 개발을 적극 권장하는 중...

# tf.estimator
tf.estimator는 PREDICT / TRAIN / EVAL 이라는 MODE에 따라 모델을 간편하게 쓸 수 있도록 지원하는 High-level API이다.

처음에는 이런저런 조건을 맞춰야해서 조금 더 헷갈릴 수 있으나 익숙해지면 오히려 골격만 맞춰주되 자잘한 부분들에 있어 상당히 편해진다

특히 모델의 graph를 저장하고 불러오는 기능을 자동으로 해주고, iteration에 따른 log를 해주는 부분이 편리하다.

In [5]:
mnist_net = tf.estimator.Estimator(model_fn=my_cnn, model_dir='tmp/mnist_net')
# tensors_to_log = {"probabilities": "softmax_tensor"}
# logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50) # Not used my simple code. Use for only log.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_task_id': 0, '_model_dir': 'tmp/mnist_net', '_session_config': None, '_master': '', '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022E52DD8668>, '_is_chief': True, '_service': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_task_type': 'worker', '_log_step_count_steps': 100}


In [6]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x=mnist.train.images, y=mnist.train.labels.astype(np.int32),
                                                    batch_size=64, num_epochs=None, shuffle=True)

In [7]:
mnist_net.train(input_fn=train_input_fn, steps=300)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/mnist_net\model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 301 into tmp/mnist_net\model.ckpt.
INFO:tensorflow:loss = 0.45215362, step = 301
INFO:tensorflow:global_step/sec: 24.1083
INFO:tensorflow:loss = 0.23355797, step = 401 (4.149 sec)
INFO:tensorflow:global_step/sec: 24.7963
INFO:tensorflow:loss = 0.29789072, step = 501 (4.033 sec)
INFO:tensorflow:Saving checkpoints for 600 into tmp/mnist_net\model.ckpt.
INFO:tensorflow:Loss for final step: 0.14420754.


In [8]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x=mnist.test.images, y=mnist.test.labels.astype(np.int32),
                                                   num_epochs=1, shuffle=False)

In [9]:
eval_results = mnist_net.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-31-11:54:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/mnist_net\model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-31-11:54:11
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.9585, global_step = 600, loss = 0.14456593
{'loss': 0.14456593, 'accuracy': 0.9585, 'global_step': 600}
